In [5]:
import torch
data = torch.rand((1024,3,128,128))*255

mean = torch.mean(data, axis=(0,2,3))
std = torch.std(data, axis=(0,2,3))
mean, std

(tensor([127.5040, 127.5023, 127.5084]), tensor([73.6087, 73.6053, 73.6131]))

In [6]:
from torchvision import transforms

custom_transforms = torch.nn.Sequential(
#     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.Normalize(mean,std),
)


data = custom_transforms(data)

mean = torch.mean(data, axis=(0,2,3))
std = torch.std(data, axis=(0,2,3))
mean, std

(tensor([-4.4950e-07, -1.7974e-07, -1.7561e-08]), tensor([1., 1., 1.]))

In [9]:
from pointnav_vo.config.vo_config.default import get_config as get_vo_config
config = get_vo_config('/datasets/home/memmel/PointNav-VO/configs/vo/vo_pointnav_main.yaml', [])
config.defrost()
config.VO.DATASET.TRAIN = config.VO.DATASET.TRAIN_WITH_NOISE
config.VO.DATASET.EVAL = config.VO.DATASET.EVAL_WITH_NOISE
config.freeze()

In [10]:
from pointnav_vo.vo import VOTransformerRegressionGeometricInvarianceEngine
engine = VOTransformerRegressionGeometricInvarianceEngine(config=config, run_type='train')
engine._set_up_dataloader(0,1)

2022-02-22 12:55:20,099 Visual Odometry configs:
BASE_TASK_CONFIG_PATH: configs/challenge_pointnav2021.local.rgbd.yaml
CHECKPOINT_FOLDER: {{LOG_DIR}}/checkpoints
DEBUG: True
ENGINE_NAME: vo_transformer_regression_geo_inv_engine
EVAL:
  EVAL_CKPT_PATH: eval_ckpt.pth
  EVAL_WITH_CKPT: True
INFO_DIR: {{LOG_DIR}}/infos
LOG_DIR: train_log/move_forward/
LOG_FILE: {{LOG_DIR}}/train.log
LOG_INTERVAL: 1
N_GPUS: -1
RESUME_STATE_FILE: resume_train_ckpt.pth
RESUME_TRAIN: False
TASK_CONFIG:
  DATASET:
    CONTENT_SCENES: ['*']
    DATA_PATH: dataset/habitat_datasets/pointnav/gibson/v2/{split}/{split}.json.gz
    SCENES_DIR: dataset/Gibson
    SPLIT: train
    TYPE: PointNav-v1
  ENVIRONMENT:
    ITERATOR_OPTIONS:
      CYCLE: True
      GROUP_BY_SCENE: True
      MAX_SCENE_REPEAT_EPISODES: -1
      MAX_SCENE_REPEAT_STEPS: 10000
      NUM_EPISODE_SAMPLE: -1
      SHUFFLE: False
      STEP_REPETITION_RANGE: 0.2
    MAX_EPISODE_SECONDS: 10000000
    MAX_EPISODE_STEPS: 500
  PYROBOT:
    BASE_CONTROLLE

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:00<00:00, 1831.75it/s]
2022-02-22 12:55:20,871 ... done. lenght 5557

2022-02-22 12:55:20,873 
Dataset: chunk bytes 180.00003814697266 MB

2022-02-22 12:55:20,873 Get index mapping from h5py ...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:00<00:00, 1743.48it/s]
2022-02-22 12:55:20,958 ... done. lenght 5435



In [19]:
# doesn't work due to train_loader length
from tqdm import tqdm

mean, std = torch.zeros(3), torch.zeros(3)

for batch in engine.train_loader:
    (data_types,
    raw_rgb_pairs,
    raw_depth_pairs,
    raw_discretized_depth_pairs,
    raw_top_down_view_pairs,
    actions,
    delta_xs,
    delta_ys,
    delta_zs,
    delta_yaws,
    dz_regress_masks,
    chunk_idxs,
    entry_idxs,) = batch
    
    rgb = torch.cat([torch.cat((pair[:,:,:,:pair.shape[-1]//2], pair[:,:,:,pair.shape[-1]//2:]),dim=0).float().to(torch.device('cpu'), non_blocking=True)
                        for pair in raw_rgb_pairs], dim=0,)
    
    mean = mean + torch.mean(rgb, axis=(0,1,2))
    std = std + torch.std(rgb, axis=(0,1,2))
    
mean = (mean / len(engine.train_loader)) / 255.
std = (std / len(engine.train_loader)) / 255.

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 33 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 34 samples have been fetched. For multiprocessing data-loading, this could be c

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 47 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 48 samples have been fetched. For multiprocessing data-loading, this could be c

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 61 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 62 samples have been fetched. For multiprocessing data-loading, this could be c

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 76 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 77 samples have been fetched. For multiprocessing data-loading, this could be c

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 91 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 92 samples have been fetched. For multiprocessing data-loading, this could be c

...

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3847 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3848 samples have been fetched. For multiprocessing data-loading, this could 

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3861 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3862 samples have been fetched. For multiprocessing data-loading, this could 

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3875 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3876 samples have been fetched. For multiprocessing data-loading, this could 

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3889 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3890 samples have been fetched. For multiprocessing data-loading, this could 

/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3903 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/memmel/miniconda/envs/pointnav-vo/lib/python3.7/site-packages/torch/utils/data/dataloader.py:447: UserWarning: Length of IterableDataset <pointnav_vo.vo.dataset.regression_geo_invariance_iter_dataset.StatePairRegressionDataset object at 0x7f720e0e7310> was reported to be 32 (when accessing len(dataloader)), but 3904 samples have been fetched. For multiprocessing data-loading, this could 

In [20]:
mean, std

(tensor([2119.0742, 1981.7532, 1896.0563]),
 tensor([792.4943, 830.8652, 909.8089]))

In [ ]:
from torchvision import transforms

normalize = torch.nn.Sequential(
    transforms.Normalize(mean,std),
)

rgb_norm = normalize(rgb)